<a href="https://colab.research.google.com/github/baeksumin/k-project/blob/main/KRX_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pickle
import json
import requests
import pandas as pd
from io import BytesIO
import pytz
from datetime import datetime, date, timedelta

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [100]:
with open("/content/drive/MyDrive/full_code.pkl","rb") as f:  
    full_code = pickle.load(f)
full_code

,full_code,short_code,codeName,marketEngName,status
0,KR7060310000,060310,3S,KOSDAQ,상장
1,KR7095570008,095570,AJ네트웍스,KOSPI,상장
2,KR7006840003,006840,AK홀딩스,KOSPI,상장
3,KR7054620000,054620,APS홀딩스,KOSDAQ,상장
4,KR7265520007,265520,AP시스템,KOSDAQ,상장
...,...,...,...,...,...
2580,KR7000542001,000547,흥국화재2우B,KOSPI,상장
2581,KR7000541003,000545,흥국화재우,KOSPI,상장
2582,KR7003280005,003280,흥아해운,KOSPI,상장
2583,KR7037440005,037440,희림,KOSDAQ,상장


In [5]:
#오늘, 어제 날짜 구하기.
def today_date():
  tz = pytz.timezone('Asia/Seoul')  #한국시간 맞추기.
  raw_dates = datetime.now(tz)
  yesterday_raw = raw_dates - timedelta(1) 
  yesterday =  yesterday_raw.strftime("%Y%m%d")  # 어제 날짜 문자열로 변환
  today = raw_dates.strftime("%Y%m%d")  # 오늘 날짜 문자열로 변환
  return yesterday, today

In [22]:
yesterday,today = today_date()
print('yesterday : '+ yesterday + '\n'+ 'today : ' + today)

yesterday : 20210928
today : 20210929


In [17]:
#  상장 폐지 종목 불러오기 (KRX_상장폐지종목현황_화면번호(20037))
def stock_delisting (yesterday, today):

  headers = {'User-Agent': 'Chrome/93.0.4577.82 Safari/537.36,'}  

  url = "http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd"

  data = {'bld': 'dbms/MDC/STAT/issue/MDCSTAT23801',
        'mktId': 'ALL',
        'tboxisuCd_finder_listdelisu0_0':'전체',
        'isuCd': 'ALL',
        'isuCd2': 'ALL',
        'strtDd': yesterday,
        'endDd': today,  
        'csvxls_isNo': 'true',}

  r = requests.post(url,data,headers=headers)
  j = json.loads(r.text)

  df = pd.json_normalize(j['output'])
  return df[['ISU_CD','ISU_NM','MKT_NM','DELIST_DD','DELIST_RSN_DSC']] #종목코드, 종목명, 시장구분, 폐지일, 폐지사유 열만 추출

In [18]:
yesterday,today = today_date() 
stock_delisting (yesterday, today) #오늘날짜 상장페지 종목 확인.

,ISU_CD,ISU_NM,MKT_NM,DELIST_DD,DELIST_RSN_DSC
0,74701772,이지스코어부동산제117호A,KOSPI,2021/09/29,수익증권 신탁기간의 만료 또는 전액 중도 환수
1,74702772,이지스코어부동산제117호F,KOSPI,2021/09/29,수익증권 신탁기간의 만료 또는 전액 중도 환수


In [15]:
# 관리종목 현황 종목 (krx화면번호_ 20012)
def issues_for_administration():
  headers = {'User-Agent': 'Chrome/93.0.4577.82 Safari/537.36,'} 

  data ={'bld': 'dbms/MDC/STAT/issue/MDCSTAT21401',
        'mktId': 'ALL',
        'csvxls_isNo': 'true',}

  url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'

  r = requests.post(url,data,headers=headers)
  j = json.loads(r.text)

  df = pd.json_normalize(j['output'])
  
  return df[['ISU_CD','ISU_NM','MKT_NM','FST_DESIGN_DD','LIST_BZ_RSN_NM']] #종목코드,종목명, 시장구분, 최초지정일, 지정사유 열만 추출.

In [37]:
issues_for_administration()

,ISU_CD,ISU_NM,MKT_NM,FST_DESIGN_DD,LIST_BZ_RSN_NM
0,131100,스카이이앤엠,KOSDAQ,2021/09/23,최근3사업연도중 2사업연도 자기자본 50%초과 법인세비용차감전계속사업손실 발생
1,317030,케이비17호스팩,KOSDAQ,2021/09/23,SPAC 상장예비심사청구서 미제출 등
2,148140,비디아이,KOSDAQ,2021/08/18,"반기검토(감사)의견 부적정, 의견거절 또는 범위제한 한정"
3,221610,자안바이오,KOSDAQ,2021/08/18,"자기자본 10억원 미만,상장폐지사유 발생,자본잠식률 50% 이상,반기검토(감사)의견..."
4,078650,지나인제약,KOSDAQ,2021/08/18,"반기검토(감사)의견 부적정, 의견거절 또는 범위제한 한정,자본잠식률 50% 이상"
...,...,...,...,...,...
109,079190,EMW,KOSDAQ,2018/10/17,상장적격성 실질심사 대상 결정
110,065150,엠피대산,KOSDAQ,2018/08/16,최근 4사업연도 연속 영업손실 발생
111,050320,아래스,KOSDAQ,2017/03/23,"상장폐지사유 발생,반기검토(감사)의견 부적정, 의견거절 또는 범위제한 한정,최근 4..."
112,115530,씨엔플러스,KOSDAQ,2017/03/16,"상장적격성 실질심사 대상 결정,사업보고서 미제출,상장폐지사유 발생"


In [103]:
# 새로운 종목 full_code 가져오기.
def KRX_search_fullcode(new_code):
  headers = {'User-Agent': 'Chrome/93.0.4577.82 Safari/537.36,'}  

  data = {'mktsel': 'ALL',
        'typeNo': '0',
        'searchText': new_code,
        'bld': 'dbms/comm/finder/finder_stkisu',}

  url = "http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd"

  r = requests.post(url,data,headers=headers)
  j = json.loads(r.text)

  df = pd.json_normalize(j['block1'])
  return df[['full_code','short_code','codeName','marketEngName']]

In [40]:
# 데이터프레임안에서 full_code 찾기
def get_fullcode(df,short_code) :
  df= df.set_index('short_code') #short code index 설정
  full_code = df.loc[short_code]['full_code']
  return full_code

In [109]:
KRX_search_fullcode('삼성전자')

,full_code,short_code,codeName,marketEngName
0,KR7005930003,005930,삼성전자,KOSPI
1,KR7005931001,005935,삼성전자우,KOSPI


In [43]:
get_fullcode(df,'238490') #위의 표 힘스(238490) full_code 가져와보기.

'KR7238490007'

In [42]:
get_fullcode(df,issues_for_administration().iloc[0,0]) #관리 종목 데이터프레임 첫번째 종목의 full_code 가져오기.

'KR7131100000'

In [97]:
# 신규상장기업현황 ( https://kind.krx.co.kr/listinvstg/listingcompany.do?method=searchListingTypeMain)

def new_listing(today): 

  headers = {'User-Agent': 'Chrome/93.0.4577.82 Safari/537.36,'} 

  data = {'method': 'searchListingTypeSub',
        'currentPageSize': '15',
        'pageIndex': '1',
        'orderMode': '1',
        'orderStat': 'D',
        'repIsuSrtCd': '',
        'isurCd': '',
        'forward': 'listingtype_sub',
        'listTypeArrStr': '01|02|03|04|05|',
        'choicTypeArrStr':'' ,
        'searchCodeType': '',
        'searchCorpName': '',
        'secuGrpArrStr': '0|ST|FS|MF|SC|RT|DR|',
        'marketType': '',
        'searchCorpNameTmp':'' ,
        'country': '410',
        'industry': '',
        'repMajAgntDesignAdvserComp': '',
        'repMajAgntComp': '',
        'designAdvserComp': '',
        'secuGrpArr': '0',
        'secuGrpArr': 'ST|FS',
        'secuGrpArr': 'MF|SC|RT',
        'secuGrpArr': 'DR',
        'listTypeArr': '01',
        'listTypeArr': '02',
        'listTypeArr': '03',
        'listTypeArr': '04',
        'listTypeArr': '05',
        'fromDate': today,
        'toDate': today,}

  url = "https://kind.krx.co.kr/listinvstg/listingcompany.do"

  r = requests.post(url,data,headers=headers)
  f = BytesIO(r.content)
  dfs = pd.read_html(f, encoding='UTF8' )
  df = dfs[0].copy()
  df = df.iloc[:,[0,1,2]]

  df =df.rename(columns = {'Unnamed: 0':'회사명','Unnamed: 1':'상장일','Unnamed: 2':'상장유형'})
  return df


In [108]:
new_stock = new_listing(today)
new_stock

,회사명,상장일,상장유형
0,실리콘투,2021-09-29,신규상장
1,에스앤디,2021-09-29,이전상장


In [122]:
new = KRX_search_fullcode(new_stock.iloc[1,0])
new.loc[0,'codeName']== new_stock.iloc[1,0]

True

In [132]:
# 신규상장 full_code krx에서 가져오기
if new_stock.shape[0] > 0:              #신규 상장이 1개라도 있으면
  for i in range(new_stock.shape[0]):    
    new_fullcode = KRX_search_fullcode(new_stock.iloc[i,0])
    print(new_fullcode)

      full_code short_code codeName marketEngName
0  KR7257720003     257720     실리콘투        KOSDAQ
      full_code short_code codeName marketEngName
0  KR7260970009     260970     에스앤디        KOSDAQ
1  KR7317400000     317400   자이에스앤디         KOSPI
